In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/MyDrive/"Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks'

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
etiketno_list = []
konum_list = []
dogruluk_orani = []

def sonucbul(path):
    for root, dirs, files in os.walk(path):
        for index,name in enumerate(files):
            if name.endswith(".jpg"):
                konum = os.path.join(path,name)
                img = cv2.imread(konum)

                img_width = img.shape[1]
                img_height = img.shape[0]
                img_blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), swapRB=True, crop=False)

                labels = ["Kidney Stone"]

                colors = ["0,255,255"]
                colors = [np.array(color.split(",")).astype("int") for color in colors]
                colors = np.array(colors)
                colors = np.tile(colors,(18,1))
                model = cv2.dnn.readNetFromDarknet("/content/drive/MyDrive/darknet/kaggle_kidney_stone_yolov4.cfg","/content/drive/MyDrive/darknet/backup/kaggle_kidney_stone_yolov4_final.weights")

                layers = model.getLayerNames()
                output_layer = [layers[layer-1] for layer in model.getUnconnectedOutLayers()]

                model.setInput(img_blob)

                detection_layers = model.forward(output_layer)


                ############## NON-MAXIMUM SUPPRESSION - OPERATION 1 ###################

                ids_list = []
                boxes_list = []
                confidences_list = []

                ############################ END OF OPERATION 1 ########################

                for detection_layer in detection_layers:
                    for object_detection in detection_layer:

                        scores = object_detection[5:]
                        predicted_id = np.argmax(scores)
                        confidence = scores[predicted_id]

                        if confidence > 0.50:

                            label = labels[predicted_id]
                            bounding_box = object_detection[0:4] * np.array([img_width,img_height,img_width,img_height])
                            (box_center_x, box_center_y, box_width, box_height) = bounding_box.astype("int")

                            start_x = int(box_center_x - (box_width/2))
                            start_y = int(box_center_y - (box_height/2))


                            ############## NON-MAXIMUM SUPPRESSION - OPERATION 2 ###################

                            ids_list.append(predicted_id)
                            confidences_list.append(float(confidence))
                            boxes_list.append([start_x, start_y, int(box_width), int(box_height)])

                            ############################ END OF OPERATION 2 ########################



                ############## NON-MAXIMUM SUPPRESSION - OPERATION 3 ###################

                max_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4)

                for max_id in max_ids:

                    max_class_id = max_id
                    box = boxes_list[max_class_id]

                    start_x = box[0] 
                    start_y = box[1] 
                    box_width = box[2] 
                    box_height = box[3] 

                    predicted_id = ids_list[max_class_id]
                    label = labels[predicted_id]
                    confidence = confidences_list[max_class_id]

                ############################ END OF OPERATION 3 ########################

                    end_x = start_x + box_width
                    end_y = start_y + box_height

                    box_color = colors[predicted_id]
                    box_color = [int(each) for each in box_color]

                    label = "{}: {:.2f}%".format(label, confidence*100)
                    konum = "{},{}-{},{}".format(start_x,start_y,end_x,end_y)
                    print("{}: {},{}-{},{} / {:.2f}".format(name,start_x,start_y,end_x,end_y,confidence*100))
                    etiketno_list.append(name)
                    konum_list.append(konum)
                    dogruluk_orani.append(confidence*100)
                    cv2.rectangle(img, (start_x,start_y),(end_x,end_y),box_color,1)
                    cv2.putText(img,label,(start_x,start_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)
                    cv2.imwrite("/content/drive/MyDrive/Model_Sonuc_Ciktilari/Kidney_Stone_KAGGLE/{}".format(name),img) #verilen path'deki tüm resimleri modele gönderdikten sonra çıkan sonucu istenilen klasöre yazdırır


In [ ]:
dosya_konumu = "/content/drive/MyDrive/darknet/kaggle_kidney_data/kidney_images_validation"

In [ ]:
sonucbul(dosya_konumu)

Stone- (1226).jpg: 352,315-360,320 / 79.92
Stone- (1323).jpg: 343,220-356,233 / 90.44
Stone- (1321).jpg: 342,221-355,233 / 96.46
Stone- (1318).jpg: 343,223-355,233 / 98.65
Stone- (1318).jpg: 391,233-399,241 / 93.04
Stone- (1305).jpg: 346,293-352,298 / 75.34
Stone- (1349).jpg: 329,278-334,283 / 89.09
Stone- (1320).jpg: 343,222-355,233 / 98.20
Stone- (1319).jpg: 343,222-355,233 / 98.41
Stone- (1319).jpg: 392,235-399,241 / 86.89
Stone- (1310).jpg: 391,231-399,238 / 81.48
Stone- (1316).jpg: 342,223-354,232 / 98.15
Stone- (1316).jpg: 390,232-400,242 / 89.04
Stone- (1311).jpg: 390,231-399,239 / 79.40
Stone- (1348).jpg: 330,278-335,284 / 94.28
Stone- (1325).jpg: 345,221-357,233 / 95.85
Stone- (1354).jpg: 336,279-342,285 / 97.03
Stone- (1326).jpg: 347,222-356,233 / 97.37
Stone- (1355).jpg: 337,280-343,284 / 89.84
Stone- (1353).jpg: 335,279-342,285 / 97.36
Stone- (1313).jpg: 343,228-350,234 / 86.15
Stone- (1313).jpg: 390,231-400,241 / 78.30
Stone- (1329).jpg: 349,223-356,232 / 93.73
Stone- (131

In [ ]:
df = pd.DataFrame({
    "Etiket Numarası" : etiketno_list,
    "Konumlar": konum_list,
    "Doğruluk oranı": dogruluk_orani
})

In [ ]:
df

,Etiket Numarası,Konumlar,Doğruluk oranı
0,Stone- (1226).jpg,"352,315-360,320",79.921526
1,Stone- (1323).jpg,"343,220-356,233",90.443373
2,Stone- (1321).jpg,"342,221-355,233",96.462500
3,Stone- (1318).jpg,"343,223-355,233",98.652184
4,Stone- (1318).jpg,"391,233-399,241",93.036067
5,Stone- (1305).jpg,"346,293-352,298",75.339961
6,Stone- (1349).jpg,"329,278-334,283",89.090121
7,Stone- (1320).jpg,"343,222-355,233",98.197293
8,Stone- (1319).jpg,"343,222-355,233",98.406410
9,Stone- (1319).jpg,"392,235-399,241",86.889392


In [ ]:
df.to_excel("/content/drive/MyDrive/Model_Sonuc_Ciktilari/Kidney_Stone_KAGGLE/sonuclar.xlsx",index=False, index_label=False)

In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks'